<a href="https://colab.research.google.com/github/simoneseverini/automated-discovery-site/blob/main/Digraphs_Hessemberg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import networkx as nx
from itertools import combinations
import math

# --- MATRIX GENERATION & FORMULAS ---

def build_givens(n, k, alpha_k):
    """Constructs the n x n Givens rotation matrix G_k(alpha_k)."""
    G = np.eye(n)
    beta_k = 1.0 - alpha_k
    G[k-1:k+1, k-1:k+1] = np.array([
        [np.sqrt(beta_k), np.sqrt(alpha_k)],
        [-np.sqrt(alpha_k), np.sqrt(beta_k)]
    ])
    return G

def build_Q_prod(n, alphas):
    """Builds Q_n by multiplying adjacent Givens rotations."""
    Q = np.eye(n)
    for k in range(1, n):
        Q = Q @ build_givens(n, k, alphas[k-1])
    return Q

def build_Q_form(n, alphas):
    """Builds Q_n using the explicit closed-form entry formula (Proposition 4)."""
    Q = np.zeros((n, n))
    betas = [1.0] + [1.0 - a for a in alphas] + [1.0]
    alphas_pad = [0.0] + list(alphas) # 1-indexed for convenience

    for i in range(1, n+1):
        for j in range(1, n+1):
            if i > j + 1:
                Q[i-1, j-1] = 0.0
            elif i == j + 1:
                Q[i-1, j-1] = -np.sqrt(alphas_pad[j])
            else:
                prod = 1.0
                for t in range(i, j):
                    prod *= np.sqrt(alphas_pad[t])
                Q[i-1, j-1] = np.sqrt(betas[i-1]) * prod * np.sqrt(betas[j])
    return Q

# --- DIGRAPH MODEL ---

def build_digraph(n, S):
    """Constructs the combinatorial support digraph D_n(S) (Theorem 9)."""
    G = nx.DiGraph()
    G.add_nodes_from(range(1, n+1))
    for k in range(1, n):
        G.add_edge(k+1, k)
    R = {1} | {k+1 for k in S}
    C = set(S) | {n}
    for i in R:
        for j in C:
            if i <= j:
                G.add_edge(i, j)
    return G

def fibonacci(k):
    if k <= 0: return 0
    if k == 1: return 1
    a, b = 0, 1
    for _ in range(2, k+1):
        a, b = b, a+b
    return b

# --- VERIFICATION SUITE ---

def run_tests():
    print("="*75)
    print(" PAPER VERIFICATION SCRIPT")
    print("="*75)

    np.random.seed(42)
    max_n = 7

    # 1. Matrix Properties
    print("\n[1] Matrix Factorization & Formula (Prop 4, Lemma 4)")
    pass_matrix = True
    for n in range(3, 8):
        alphas = np.random.uniform(0.1, 0.9, n-1)
        Q_p = build_Q_prod(n, alphas)
        Q_f = build_Q_form(n, alphas)

        ortho = np.allclose(Q_p.T @ Q_p, np.eye(n))
        hess = np.allclose(np.tril(Q_p, -2), 0)
        form = np.allclose(Q_p, Q_f)
        if not (ortho and hess and form):
            pass_matrix = False
    print(f" -> Q is orthogonal, upper Hessenberg, and formula matches: {'PASS' if pass_matrix else 'FAIL'}")

    # 2. Block disconnection
    print("\n[2] Connectivity (Lem 7, Thm 9)")
    alphas_break = [0.5, 0.0, 0.5, 0.5, 0.5] # n=6, break at alpha_2
    Q_break = build_Q_form(6, alphas_break)
    D_break = nx.DiGraph(np.abs(Q_break) > 1e-10)
    pass_break = not nx.is_weakly_connected(D_break)

    pass_model = True
    for S in combinations(range(1, 6), 3):
        alphas = [0.5 if k in S else 1.0 for k in range(1, 6)]
        Q = build_Q_form(6, alphas)
        D_Q = nx.DiGraph(np.abs(Q) > 1e-10)
        # Fix node labels from 0-indexed to 1-indexed for comparison
        D_Q = nx.relabel_nodes(D_Q, {i: i+1 for i in range(6)})
        D_S = build_digraph(6, S)
        if set(D_Q.edges()) != set(D_S.edges()):
            pass_model = False

    print(f" -> Lemma 7 (Break disconnects): {'PASS' if pass_break else 'FAIL'}")
    print(f" -> Theorem 9 (Combinatorial model exactly matches D(Q)): {'PASS' if pass_model else 'FAIL'}")

    # 3. Graph structural properties
    print("\n[3] Structural Properties (Lem 10, Prop 11, Thm 12)")
    pass_hc, pass_deg, pass_rigid, pass_singleton = True, True, True, True

    n = 6
    for r in range(n):
        for S_tup in combinations(range(1, n), r):
            S = set(S_tup)
            m = len(S)
            G = build_digraph(n, S)

            # HC
            cycles = list(nx.simple_cycles(G))
            if len([c for c in cycles if len(c) == n]) != 1: pass_hc = False

            # Degrees
            C = S | {n}
            for i in range(1, n+1):
                # Out
                if i == 1: exp_out = m + 1
                elif (i-1) not in S: exp_out = 1
                else: exp_out = 1 + len([c for c in C if c >= i])
                if G.out_degree(i) != exp_out: pass_deg = False

                # In
                if i < n and i not in S: exp_in = 1
                elif i in S: exp_in = 2 + len([s for s in S if s <= i-1])
                elif i == n: exp_in = m + 1
                if G.in_degree(i) != exp_in: pass_deg = False

            # Automorphism
            if m >= 2:
                matcher = nx.algorithms.isomorphism.DiGraphMatcher(G, G)
                if len(list(matcher.isomorphisms_iter())) != 1: pass_rigid = False

    for t in range(1, n):
        G1 = build_digraph(n, {t})
        G2 = build_digraph(n, {n-t})
        if not nx.is_isomorphic(G1, G2): pass_singleton = False

    print(f" -> Lemma 10 (Unique Hamilton Cycle): {'PASS' if pass_hc else 'FAIL'}")
    print(f" -> Proposition 11 (Exact Degrees): {'PASS' if pass_deg else 'FAIL'}")
    print(f" -> Theorem 12 (Trivial automorphism for |S|>=2): {'PASS' if pass_rigid else 'FAIL'}")
    print(f" -> Lemma 13 (Singleton symmetry): {'PASS' if pass_singleton else 'FAIL'}")

    # 4. Enumeration
    print("\n[4] Enumerations (Thm 14, Cor 16, Thm 17)")
    print(f"  {'n':<3} | {'Conn Act':<10} | {'Conn Theo':<10} | {'Loop Act':<10} | {'Loop Theo':<10}")
    print("  " + "-"*50)

    pass_enum = True
    for n_test in range(3, 8):
        graphs, loopless = [], []
        for r in range(n_test):
            for S_tup in combinations(range(1, n_test), r):
                S = set(S_tup)
                G = build_digraph(n_test, S)
                graphs.append((S, G))

                # Check for loops
                has_loop = any(G.has_edge(v,v) for v in G.nodes())
                if not has_loop:
                    loopless.append(G)

        def count_iso(g_list):
            classes = []
            for item in g_list:
                g = item[1] if isinstance(item, tuple) else item
                if not any(nx.is_isomorphic(g, u) for u in classes):
                    classes.append(g)
            return len(classes)

        act_conn = count_iso(graphs)
        act_loop = count_iso(loopless)

        theo_conn = (2**(n_test-1)) - ((n_test-1)//2)
        theo_loop = fibonacci(n_test-1) - ((n_test-3)//2)

        if act_conn != theo_conn or act_loop != theo_loop: pass_enum = False

        print(f"  {n_test:<3} | {act_conn:<10} | {theo_conn:<10} | {act_loop:<10} | {theo_loop:<10}")

    print(f" -> Enumeration Theorems Match: {'PASS' if pass_enum else 'FAIL'}")

    # 5. Example Matrices
    print("\n[5] Examples Match")
    # Ex 1: n=5, S={2}
    Q18 = build_Q_form(5, [1.0, 0.5, 1.0, 1.0])
    exp_Q18 = np.array([
        [0, 1/np.sqrt(2), 0, 0, 1/np.sqrt(2)],
        [-1, 0, 0, 0, 0],
        [0, -1/np.sqrt(2), 0, 0, 1/np.sqrt(2)],
        [0, 0, -1, 0, 0],
        [0, 0, 0, -1, 0]
    ])
    pass_ex18 = np.allclose(Q18, exp_Q18)

    # Ex 2: n=5, S={1,3}
    Q19 = build_Q_form(5, [0.5, 1.0, 0.5, 1.0])
    exp_Q19 = np.array([
        [1/np.sqrt(2), 0, 0.5, 0, 0.5],
        [-1/np.sqrt(2), 0, 0.5, 0, 0.5],
        [0, -1, 0, 0, 0],
        [0, 0, -1/np.sqrt(2), 0, 1/np.sqrt(2)],
        [0, 0, 0, -1, 0]
    ])
    pass_ex19 = np.allclose(Q19, exp_Q19)
    print(f" -> Explicit Matrices match Example output: {'PASS' if pass_ex18 and pass_ex19 else 'FAIL'}")

    print("\n" + "="*75)
    if pass_matrix and pass_break and pass_model and pass_hc and pass_deg and pass_rigid and pass_singleton and pass_enum and pass_ex18 and pass_ex19:
        print(" SUCCESS: ALL PAPER CLAIMS RIGOROUSLY VERIFIED BY COMPUTATION.")
    else:
        print(" FAILURE: SOME TESTS DID NOT PASS.")
    print("="*75)

if __name__ == '__main__':
    run_tests()

 PAPER VERIFICATION SCRIPT

[1] Matrix Factorization & Formula (Prop 4, Lemma 4)
 -> Q is orthogonal, upper Hessenberg, and formula matches: PASS

[2] Connectivity (Lem 7, Thm 9)
 -> Lemma 7 (Break disconnects): PASS
 -> Theorem 9 (Combinatorial model exactly matches D(Q)): PASS

[3] Structural Properties (Lem 10, Prop 11, Thm 12)
 -> Lemma 10 (Unique Hamilton Cycle): PASS
 -> Proposition 11 (Exact Degrees): PASS
 -> Theorem 12 (Trivial automorphism for |S|>=2): PASS
 -> Lemma 13 (Singleton symmetry): PASS

[4] Enumerations (Thm 14, Cor 16, Thm 17)
  n   | Conn Act   | Conn Theo  | Loop Act   | Loop Theo 
  --------------------------------------------------
  3   | 3          | 3          | 1          | 1         
  4   | 7          | 7          | 2          | 2         
  5   | 14         | 14         | 2          | 2         
  6   | 30         | 30         | 4          | 4         
  7   | 61         | 61         | 6          | 6         
 -> Enumeration Theorems Match: PASS

[5] Ex